# Random Forest Regression from Scratch

Group 18 Members:

- Clara Pichler, 11917694
- Hannah Knapp, 11901857 
- Sibel Toprakkiran, 09426341

### Overview

1. Bootstraping
- `make_bootstraps(X, y, n_bootstraps=100)`

2. Decision Tree Regression
- metrcis
- `split_dataset(X, y, feature_idx, threshold)`
- `find_best_split(X, y)`
- `build_tree(X, y, max_depth, min_samples_split, depth=0)`
- `predict_tree(tree, X)`

3. Random Forest Regression

4. Random Forest Regression - LLM (ChatGPT)

4. K-NN Regression

5. Evaluation
- Ours
- LLM
- sklearn
- k-nn

In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

We will use the data sets `airfoil_noise_data.csv` and `abalone.csv` as data frames for testing our implementation.

In [30]:
df_airfol = pd.read_csv("./data/airfoil_noise_data.csv")

display(df_airfol.head())
display(df_airfol.info())

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      1503 non-null   int64  
 1   x1      1503 non-null   float64
 2   x2      1503 non-null   float64
 3   x3      1503 non-null   float64
 4   x4      1503 non-null   float64
 5   y       1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


None

In [31]:
column_names = ["Sex", "Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight", "Rings"]
df_abalone = pd.read_csv("./data/abalone.csv", header=0, names=column_names)
df_abalone[df_abalone.Height == 0]
df_abalone = df_abalone[df_abalone.Height != 0]
df_abalone = pd.get_dummies(df_abalone, columns=['Sex'], drop_first=False)

display(df_abalone.head())
display(df_abalone.info())

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,False,False,True
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,False,False,True
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,True,False,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,False,False,True
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,True,False


<class 'pandas.core.frame.DataFrame'>
Index: 4175 entries, 0 to 4176
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          4175 non-null   float64
 1   Diameter        4175 non-null   float64
 2   Height          4175 non-null   float64
 3   Whole_weight    4175 non-null   float64
 4   Shucked_weight  4175 non-null   float64
 5   Viscera_weight  4175 non-null   float64
 6   Shell_weight    4175 non-null   float64
 7   Rings           4175 non-null   int64  
 8   Sex_F           4175 non-null   bool   
 9   Sex_I           4175 non-null   bool   
 10  Sex_M           4175 non-null   bool   
dtypes: bool(3), float64(7), int64(1)
memory usage: 305.8 KB


None

## Bootstrapping

Bootstrapping is a method to create multiple subsets of the original data by sampling with replacement. Each subset is used to train one decision tree in the forest.

- Introduces randomness, ensuring that trees see different views of the data.
- Helps in reducing overfitting by decorrelating the trees.

- __Sample Size__: The size of the bootstrap sample is the same as the original dataset.
- __Replacement__: Sampling with replacement ensures diversity between bootstrapped samples.


In [32]:
def make_bootstraps(X, y, n_bootstraps=100):
    
    bootstrap = []
    sample_size = X.shape[0]
    idx = np.arange(sample_size)

    for b in range(n_bootstraps):
        
        sidx = np.random.choice(idx, size=sample_size, replace=True)
        X_boot = X[sidx]
        y_boot = y[sidx]
        
        bootstrap.append((X_boot, y_boot))
        
    return bootstrap

## Decision Tree Regression

First, we define some helper functions to split the data and calculate metrics like the mean squared error (MSE) to figure out the best split.
And for the actual building of the trees we recursively split the data into smaller groups, based on feature thresholds, until a stopping condition is met (e.g., max depth or minimum samples per leaf).


In [33]:
def mse(y):
    return np.mean((y - np.mean(y))**2)

def mse_split(parent, y_left, y_right):

    total = len(parent)
    weighted_mse = (len(y_left) * mse(y_left) + len(y_right) * mse(y_right)) / total

    return weighted_mse


def variance_reduction_split(parent, y_left, y_right):

    total = len(parent)
    parent_variance = np.var(parent)
    weighted_variance = (len(y_left) * np.var(y_left) + len(y_right) * np.var(y_right)) / total

    return parent_variance - weighted_variance


def mae_split(parent, y_left, y_right):

    total = len(parent)
    weighted_mae = (len(y_left) * np.mean(np.abs(y_left - np.mean(y_left))) +
                    len(y_right) * np.mean(np.abs(y_right - np.mean(y_right)))) / total
    
    return weighted_mae


In [34]:
metric_functions = {
        "mse": mse_split,
        "variance_reduction": variance_reduction_split,
        "mae": mae_split,
}

__Mean Squared Error (MSE)__
$$
\text{MSE}(y) = \frac{1}{n} \sum_{i=1}^n (y_i - \bar{y})^2
$$ 
Weighted Split Formula:
$$
\frac{|y_{\text{left}}| \cdot \text{MSE}(y_{\text{left}}) + |y_{\text{right}}| \cdot \text{MSE}(y_{\text{right}})}{|y|}
$$  

- MSE minimizes the squared differences between actual and predicted values
- It works well if the target variable is continuous 
- Sensitive to outliers since squared errors penalize large deviations more heavily 
- We will choose the split with the lowest weighted MSE!



__Variance Reduction__
 
$$
\text{Var}(y) - \left( \frac{|y_{\text{left}}| \cdot \text{Var}(y_{\text{left}}) + |y_{\text{right}}| \cdot \text{Var}(y_{\text{right}})}{|y|} \right)
$$ 
- Measures the decrease in variance after splitting.  
- Effective  when capturing spread or variability in the target variable is important.  
- Generally, less sensitive to outliers compared to MSE.
- Choosing the split with the highest variance reduction!


__Mean Absolute Error (MAE)__

$$
\text{MAE}(y) = \frac{1}{n} \sum_{i=1}^n |y_i - \bar{y}|
$$ 
Weighted Split Formula: 
$$
\frac{|y_{\text{left}}| \cdot \text{MAE}(y_{\text{left}}) + |y_{\text{right}}| \cdot \text{MAE}(y_{\text{right}})}{|y|}
$$  
- MAE minimizes the __absolute__ differences between actual and predicted values.  
- Works well when dealing with data containing outliers, as it penalizes them less compared to MSE.  
- We will choose the split with the lowest weighted MAE!


In [35]:
def split_dataset(X, y, feature_idx, threshold):
    
    left_mask = X[:, feature_idx] <= threshold
    right_mask = ~left_mask

    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

`split_dataset` divides X (features) and y (target) into two groups based on whether the value of a feature is less than or equal to a given threshold. This is used to evaluate potential splits during training. For example split on feature `feature_1` at value 6 creates two groups: rows where `feature_1 <= 6` and rows where `feature_1 > 6`.

In [36]:
def find_best_split(X, y, feature_subset=None, metric="mse"):
    best_feature, best_threshold = None, None

    if metric in ["variance_reduction", "information_gain"]:
        best_metric = float("-inf")  
        is_better = lambda current, best: current > best
    else:
        best_metric = float("inf")  
        is_better = lambda current, best: current < best

    features = feature_subset if feature_subset is not None else range(X.shape[1])

    for feature_idx in features:
        thresholds = np.unique(X[:, feature_idx])

        for threshold in thresholds:
            _, _, y_left, y_right = split_dataset(X, y, feature_idx, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            metric_value = metric_functions[metric](y, y_left, y_right)

            if is_better(metric_value, best_metric):
                best_metric = metric_value
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold

`find_best_split` evaluates all possible splits for every feature and every threshold. It selects the best split based on the chosen metric

It evaluates how well the splits separate the data into subsets that are better at differantiating the target value and making the leaf nodes more pure.

In [37]:
def build_tree(X, y, max_depth, min_samples_split, max_features, depth=0, metric="mse"):

    if depth >= max_depth or len(y) < min_samples_split or mse(y) == 0:
        return np.mean(y)  

    n_features = X.shape[1]
    if max_features == 'sqrt':
        feature_subset = np.random.choice(n_features, size=max(1, int(np.sqrt(n_features))), replace=False)
    elif max_features == 'log2':
        feature_subset = np.random.choice(n_features, size=max(1, int(np.log2(n_features))), replace=False)
    else:
        feature_subset = np.arange(n_features)
    
    feature_idx, threshold = find_best_split(X, y, feature_subset, metric=metric)
    
    if feature_idx is None:
        return np.mean(y)  
    
    X_left, X_right, y_left, y_right = split_dataset(X, y, feature_idx, threshold)
    left_subtree = build_tree(X_left, y_left, max_depth, min_samples_split, max_features, depth + 1, metric)
    right_subtree = build_tree(X_right, y_right, max_depth, min_samples_split, max_features, depth + 1, metric)
    
    return {
        "feature_idx": feature_idx,
        "threshold": threshold,
        "left": left_subtree,
        "right": right_subtree,
    }


- __Stopping Conditions__: Stops if the max depth is reached, if there are fewer samples than min_samples_split, or if the MSE is 0 (all values are the same).
- __Recursive Splitting__: For each split, the function creates a left and right subtree until the stopping conditions are met.
- __Leaf Node__: If the recursion stops, the tree stores the mean value of y for prediction.
- __Feature Selection__: Chooses $\sqrt{n}$ features randomly for each tree, where $n$ is the number of original features

In [38]:
def predict_tree(tree, X):

    if isinstance(tree, dict):

        feature_idx = tree["feature_idx"]
        threshold = tree["threshold"]

        if X[feature_idx] <= threshold:
            return predict_tree(tree["left"], X)
        
        else:
            return predict_tree(tree["right"], X)
        
    else:
        return tree  # Leaf node


Traverses the tree based on the input features until you reach a leaf node.
Returns the mean value of the target variable y at the leaf node.

## Random Forest Regression

In [39]:
class RandomForestRegressor_18:
    
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=10, max_features='sqrt', metric='mse'):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.metric = metric
        self.max_features = max_features
        self.trees = []
        
    def fit(self, X, y):
        bootstraps = make_bootstraps(X, y, n_bootstraps=self.n_trees)
        self.trees = [
            build_tree(X_boot, y_boot, self.max_depth, self.min_samples_split, self.max_features, metric=self.metric)
                for X_boot, y_boot in bootstraps
            ]
    
    def predict(self, X):
        predictions = np.array([predict_tree(tree, x) for tree in self.trees for x in X])
        predictions = predictions.reshape(len(self.trees), len(X))

        return np.mean(predictions, axis=0)

## Evaluation

We use for the comparison of the methods the same holdout split with the training set containg 75% of the data.

In [40]:
X = df_airfol.drop("y", axis=1).values
y = df_airfol["y"].values

#X = df_abalone.drop("Rings", axis=1).values
#y = df_abalone["Rings"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

For evaluation we use the following three methods:

__Mean Squared Error (MSE)__

The average of the squared differences between the predicted values and the actual values. It gives more weight to larger errors.

- A smaller MSE value indicates that the model’s predictions are close to the actual values.
- Since it's based on squared differences, large prediction errors (outliers) have a greater impact.
- MSE is in the square of the unit of your target variable
- Useful when large errors are particularly undesirable and need to be penalized more heavily.



__Mean Absolute Error (MAE)__

The average of the absolute differences between predicted values and actual values. Unlike MSE, it treats all errors equally, regardless of size.

- A smaller MAE value indicates better model performance.
- MAE is in the same unit as the target variable, making it more interpretable compared to MSE.
- Good for understanding the typical size of prediction errors.
- Less sensitive to outliers compared to MSE.

__R-squared__

The proportion of variance in the target variable that the model explains. It ranges from:
- __1__: Perfect fit (model explains all variance in the data).
- __0__: Model does no better than predicting the mean of the target.
- __Negative__: Model performs worse than simply predicting the mean.


- A higher R-squared (close to 1) indicates a good fit.
- A low or negative R-squared suggests that your model is not capturing the relationship between the features and target effectively.
- Helps understand how well the model explains the variability in the target variable.
- Not ideal for measuring absolute error but useful for comparing models.

### Our Regressor

In [45]:
rf_18 = RandomForestRegressor_18(n_trees=150, max_depth=10, min_samples_split=2, max_features='sqrt', metric='variance_reduction')
rf_18.fit(X_train, y_train)

predictions_18 = rf_18.predict(X_test)

In [46]:
mse_18 = mean_squared_error(y_test, predictions_18)
mae_18 = mean_absolute_error(y_test, predictions_18)
r2_18 = r2_score(y_test, predictions_18)

print("Model Performance:")
print(f"Mean Squared Error: {mse_18:.2f}")
print(f"Mean Absolute Error: {mae_18:.2f}")
print(f"R-squared: {r2_18:.2f}")

Model Performance:
Mean Squared Error: 6.23
Mean Absolute Error: 1.92
R-squared: 0.87


### ChatGPT Regressor

In [83]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        self.tree = self._build_tree(data)

    def predict(self, X):
        return np.array([self._predict_row(row, self.tree) for row in X])

    def _build_tree(self, data, depth=0):
        X, y = data[:, :-1], data[:, -1]
        num_samples, num_features = X.shape

        # Stopping conditions
        if num_samples < self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            return np.mean(y)

        feature_idx, threshold = self._find_best_split(X, y)
        if feature_idx is None:
            return np.mean(y)

        left_idxs, right_idxs = self._split(X[:, feature_idx], threshold)
        left = self._build_tree(data[left_idxs], depth + 1)
        right = self._build_tree(data[right_idxs], depth + 1)
        return {"feature_idx": feature_idx, "threshold": threshold, "left": left, "right": right}

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_mse = float("inf")
        split_idx, split_thresh = None, None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                mse = self._mean_squared_error_split(X[:, feature_idx], y, threshold)
                if mse < best_mse:
                    best_mse, split_idx, split_thresh = mse, feature_idx, threshold

        return split_idx, split_thresh

    def _mean_squared_error_split(self, feature, y, threshold):
        left_idxs, right_idxs = self._split(feature, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return float("inf")

        y_left, y_right = y[left_idxs], y[right_idxs]
        mse_left = np.mean((y_left - np.mean(y_left))**2)
        mse_right = np.mean((y_right - np.mean(y_right))**2)

        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        mse = (n_left / n) * mse_left + (n_right / n) * mse_right

        return mse

    def _split(self, feature, threshold):
        left_idxs = np.where(feature <= threshold)[0]
        right_idxs = np.where(feature > threshold)[0]
        return left_idxs, right_idxs

    def _predict_row(self, row, tree):
        if isinstance(tree, dict):
            if row[tree["feature_idx"]] <= tree["threshold"]:
                return self._predict_row(row, tree["left"])
            else:
                return self._predict_row(row, tree["right"])
        return tree


class RFRegressor:
    def __init__(self, n_trees=10, max_depth=None, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples, n_features = X.shape
        for _ in range(self.n_trees):
            bootstrap_idxs = np.random.choice(n_samples, n_samples, replace=True)
            bootstrap_X = X[bootstrap_idxs]
            bootstrap_y = y[bootstrap_idxs]

            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            if self.max_features is not None:
                feature_idxs = np.random.choice(n_features, self.max_features, replace=False)
                tree.fit(bootstrap_X[:, feature_idxs], bootstrap_y)
                tree.feature_idxs = feature_idxs
            else:
                tree.fit(bootstrap_X, bootstrap_y)
                tree.feature_idxs = None
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X[:, tree.feature_idxs] if tree.feature_idxs is not None else X)
                                      for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

In [84]:
rf_chatgpt = RFRegressor(n_trees=150, max_depth=10, min_samples_split=2)
rf_chatgpt.fit(X_train, y_train)
predictions_chatgpt = rf_chatgpt.predict(X_test)

In [85]:
mse_chatgpt = mean_squared_error(y_test, predictions_chatgpt)
mae_chatgpt = mean_absolute_error(y_test, predictions_chatgpt)
r2_chatgpt = r2_score(y_test, predictions_chatgpt)

print("Model Performance:")
print(f"Mean Squared Error: {mse_chatgpt:.2f}")
print(f"Mean Absolute Error: {mae_chatgpt:.2f}")
print(f"R-squared: {r2_chatgpt:.2f}")

Model Performance:
Mean Squared Error: 4.40
Mean Absolute Error: 1.54
R-squared: 0.91


### SkLearn Regressor

In [86]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [87]:
rf_sklearn = RandomForestRegressor(n_estimators=50, max_depth=10, min_samples_split=2)
rf_sklearn.fit(X_train, y_train)

predictions_sklearn = rf_sklearn.predict(X_test)

In [88]:
param_grid = {
    "n_estimators": [10, 25, 50, 100, 150],
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 3, 3, 4],
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
print(best_rf)
best_rf.fit(X_train, y_train)
best_predictions = best_rf.predict(X_test)

RandomForestRegressor(max_depth=15)


In [89]:
mse_sklearn = mean_squared_error(y_test, predictions_sklearn)
mae_sklearn = mean_absolute_error(y_test, predictions_sklearn)
r2_sklearn = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_sklearn:.2f}")
print(f"Mean Absolute Error: {mae_sklearn:.2f}")
print(f"R-squared: {r2_sklearn:.2f}")

Model Performance:
Mean Squared Error: 4.15
Mean Absolute Error: 1.52
R-squared: 0.91


In [90]:
mse_sklearn_best = mean_squared_error(y_test, best_predictions)
mae_sklearn_best = mean_absolute_error(y_test, best_predictions)
r2_sklearn_best = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_sklearn_best:.2f}")
print(f"Mean Absolute Error: {mae_sklearn_best:.2f}")
print(f"R-squared: {r2_sklearn:.2f}")

Model Performance:
Mean Squared Error: 3.32
Mean Absolute Error: 1.33
R-squared: 0.91


### K-NN Regressor

In [91]:
from sklearn.neighbors import KNeighborsRegressor

knn_sklearn = KNeighborsRegressor(n_neighbors=5)
knn_sklearn.fit(X_train, y_train)

predictions_sklearn = knn_sklearn.predict(X_test)

In [92]:
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

knn_sklearn = KNeighborsRegressor()
grid_search = GridSearchCV(knn_sklearn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

predictions_sklearn_best = grid_search.predict(X_test)

In [93]:
mse_knn = mean_squared_error(y_test, predictions_sklearn)
mae_knn = mean_absolute_error(y_test, predictions_sklearn)
r2_knn = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_knn:.2f}")
print(f"Mean Absolute Error: {mae_knn:.2f}")
print(f"R-squared: {r2_knn:.2f}")

Model Performance:
Mean Squared Error: 38.82
Mean Absolute Error: 4.95
R-squared: 0.19


In [94]:
mse_knn_best = mean_squared_error(y_test, predictions_sklearn_best)
mae_knn_best = mean_absolute_error(y_test, predictions_sklearn_best)
r2_knn_best = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_knn_best:.2f}")
print(f"Mean Absolute Error: {mae_knn_best:.2f}")
print(f"R-squared: {r2_knn_best:.2f}")

Model Performance:
Mean Squared Error: 37.86
Mean Absolute Error: 4.86
R-squared: 0.19


## Notes

Max Depth: Prevents trees from growing too deep, which could lead to overfitting.

Min Samples Split: Controls the smallest group size allowed for further splitting, preventing unnecessary splits.


**Key Insights from These Metrics**
1. **MSE**:
   - If it's high, your model is making some large errors that need to be addressed.
   - If it's low, your model is capturing most of the relationship.

2. **MAE**:
   - Directly tells you the average prediction error. 
   - Compare it to the scale of your target variable; if MAE is relatively low, the model is performing well.

3. **R²**:
   - A high R² suggests the model explains a significant portion of the target variable's variance.
   - If R² is low (or negative), consider if the features are truly predictive of the target or if the model is too simple/complex.


**Next Steps Based on Metrics**

- **High MSE or MAE**:
  - Investigate outliers, or whether the model needs better hyperparameter tuning.
  - Consider adding more predictive features or improving feature engineering.

- **Low R²**:
  - Evaluate if features are relevant or add more features to capture the variance.
  - Consider if the model is underfitting or overfitting:
    - Underfitting: Increase `max_depth`, add more `n_trees`.
    - Overfitting: Decrease `max_depth` or regularize.
